<a href="https://colab.research.google.com/github/hjb-001/GeminiChatUp/blob/main/Embedding/PyTorch/Advanced/ProtT5-XL-UniRef50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Important Notes:
1. ProtT5-XL-UniRef50 has both encoder and decoder, for feature extraction we only load and use the encoder part.
2. Loading only the encoder part, reduces the inference speed and the GPU memory requirements by half.
2. In order to use ProtT5-XL-UniRef50 encoder, you must install the latest huggingface transformers version from their GitHub repo.
3. If you are intersted in both the encoder and decoder, you should use T5Model rather than T5EncoderModel.

<h3>Extracting protein sequences' features using ProtT5-XL-UniRef50 pretrained-model</h3>

**1. Load necessry libraries including huggingface transformers**

In [1]:
!pip install -q SentencePiece transformers

In [2]:
import torch
from transformers import T5EncoderModel, T5Tokenizer
import re
import numpy as np
import gc

<b>2. Load the vocabulary and ProtT5-XL-UniRef50 Model<b>

In [3]:
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")

pytorch_model.bin:   0%|          | 0.00/11.3G [00:00<?, ?B/s]

In [16]:
gc.collect()

60

<b>3. Load the model into the GPU if avilabile and switch to inference mode<b>

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [7]:
model = model.to(device)
model = model.eval()

<b>4. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [8]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [9]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>5. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [10]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, padding=True)

In [11]:
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)

<b>6. Extracting sequences' features and load it into the CPU if needed<b>

In [12]:
with torch.no_grad():
    embedding = model(input_ids=input_ids,attention_mask=attention_mask)

In [13]:
embedding = embedding.last_hidden_state.cpu().numpy()

<b>7. Remove padding (\<pad\>) and special tokens (\</s\>) that is added by ProtT5-XL-UniRef50 model<b>

In [14]:
features = []
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    seq_emd = embedding[seq_num][:seq_len-1]
    features.append(seq_emd)

In [18]:
print(features)
print(len(features[0]))

[array([[ 0.17167199, -0.14080104, -0.20465545, ...,  0.1451706 ,
         0.1475328 , -0.07327209],
       [ 0.09367213, -0.1169606 , -0.29974896, ...,  0.10004929,
        -0.22623882,  0.22561264],
       [ 0.19351341, -0.09518166, -0.29209894, ...,  0.06696439,
         0.03064489,  0.13165711],
       ...,
       [ 0.49161172,  0.0009896 , -0.18922976, ...,  0.08726893,
         0.49303588,  0.14975205],
       [ 0.07876217, -0.07448191, -0.19791865, ..., -0.07003246,
         0.09184478, -0.24240705],
       [ 0.45733184,  0.004346  , -0.28679514, ..., -0.0081778 ,
         0.2877469 , -0.01780241]], dtype=float32), array([[ 0.16555823, -0.09297126, -0.22606148, ..., -0.07201444,
        -0.11815777,  0.15539685],
       [ 0.11265311, -0.12298614, -0.11732373, ...,  0.05776759,
        -0.3005794 ,  0.19830094],
       [ 0.3078941 , -0.10488591, -0.16040765, ..., -0.06531574,
         0.10468011, -0.07694175],
       [ 0.3379021 , -0.20987591, -0.2879338 , ...,  0.03746886,
     